In [1]:
import os

In [1]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

### Clone Github Repository

In [2]:
%pwd

'c:\\GenerativeAI-Basics\\Project-Generative-AI\\source-code-analysis-project-GenAl\\research'

In [3]:
!mkdir test_repo

In [5]:
repo_path = "test_repo/"
Repo.clone_from("https://github.com/VaibhavBomle/Llama-2-Project-on-CPU",to_path=repo_path)

<git.repo.base.Repo 'c:\\GenerativeAI-Basics\\Project-Generative-AI\\source-code-analysis-project-GenAl\\research\\test_repo\\.git'>